In [1]:
from tensorflow.keras import backend as K
import numpy as np 
import os
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
import pathlib as pl
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

### utilities -- needed for model, specified for training

In [5]:
def dice_coef(truth, pred, smooth=1):
    
    true_flat = tf.reshape(truth, [-1])
    pred_flat = tf.reshape(pred, [-1])
    
    intersection = tf.reduce_sum(true_flat*pred_flat)
    return ((2. * intersection)+smooth) / (tf.reduce_sum(true_flat)+tf.reduce_sum(pred_flat) + smooth)


def dice_coef_loss(gt, pred):
    return 1-dice_coef(gt, pred)

### basic unet code

In [6]:
# shrink the size of the convolution layers
shrink = 8

In [7]:
def unet(pretrained_weights = None,input_size = (240,240,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512/shrink, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256/shrink, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128/shrink, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64/shrink, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64/shrink, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss =dice_coef_loss, metrics = [dice_coef])

    return model


# Create model

In [8]:
model = unet()

# load weights from path

In [13]:
model.load_weights("extra_stuff/weights00000200.h5")

# get all weights for all layers

In [40]:
#model.get_weights()

# get weights for a single layer

- input layer has no weights

In [41]:
model.layers[0].get_weights()

[]

### get weights for the 1st conv layer

- example conv layer
- weights come in a list

In [42]:
model.layers[1].get_weights()

[array([[[[ 0.1152328 ,  0.5276617 , -0.7377614 , -0.52538556,
            0.45505467,  0.7270211 , -0.38682103, -0.00731626]],
 
         [[-0.568643  ,  0.2057117 ,  0.15583187, -0.38538706,
            0.17416316,  0.8215804 , -0.16912712, -0.20825058]],
 
         [[ 0.2531662 ,  0.27577066, -0.408096  , -0.08812112,
            0.12006578,  0.10969435,  0.23568417, -0.05589726]]],
 
 
        [[[-0.4871996 , -0.09196077, -0.4233251 ,  0.16528891,
            0.27917728,  0.9409635 ,  0.29363814,  0.8791226 ]],
 
         [[ 0.53708804, -0.36101764,  0.40414205, -0.1210577 ,
            0.45317525, -0.22057651,  0.6186366 , -0.11786763]],
 
         [[-0.1941728 , -0.2505122 ,  0.6309221 , -0.00540727,
           -0.5391329 ,  0.56538314, -0.7884627 ,  0.17512968]]],
 
 
        [[[ 0.52644366,  0.42978835,  0.0324479 ,  0.53517   ,
            0.20992978, -1.0285648 , -0.99992865,  0.10883015]],
 
         [[ 0.23594022, -0.34976366, -0.0488033 ,  0.6410055 ,
            0.2480007

# 2 element list
- let's look through these

In [43]:
len( model.layers[1].get_weights() )

2

### 0th idx

- 3x3 kernel size for the 64/shrink=8 filters in the first conv layer. and the 1 corresponds to 1 channel?

In [46]:
model.layers[1].get_weights()[0].shape

(3, 3, 1, 8)

### 1st idx

- 8 biases for the 8 filters?

In [47]:
model.layers[1].get_weights()[1].shape

(8,)